Для начала импортируем все необходимые нам библиотеки и данные

Устанавливаем библиотеку surprise, если у вас её нет:

In [54]:
#!pip install surprise

In [55]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [56]:
movies = pd.read_csv('C:\SkillFactory\SF_DataScience\Current_tasks\data\DS_ADD_1\movies.csv')
ratings = pd.read_csv('C:\SkillFactory\SF_DataScience\Current_tasks\data\DS_ADD_1\mratings.csv')

Наши данные содержат информацию о фильмах и их жанрах:

In [57]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


А также информацию о том, какие рейтинги поставили пользователи фильмам:

In [58]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Для удобства объединим данные из двух табличек в одну и почистим недостающие данные (пропуски в потенциальной таблице user-item):

In [59]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [60]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


Вот так мы можем по текущей таблице посмотреть, например, какие фильмы смотрел пользователь с id=2.0:

In [61]:
movies_with_ratings[movies_with_ratings.userId == 2.0].title.unique()

array(['Shawshank Redemption, The (1994)', 'Tommy Boy (1995)',
       'Good Will Hunting (1997)', 'Gladiator (2000)',
       'Kill Bill: Vol. 1 (2003)', 'Collateral (2004)',
       'Talladega Nights: The Ballad of Ricky Bobby (2006)',
       'Departed, The (2006)', 'Dark Knight, The (2008)',
       'Step Brothers (2008)', 'Inglourious Basterds (2009)',
       'Zombieland (2009)', 'Shutter Island (2010)',
       'Exit Through the Gift Shop (2010)', 'Inception (2010)',
       'Town, The (2010)', 'Inside Job (2010)',
       'Louis C.K.: Hilarious (2010)', 'Warrior (2011)',
       'Dark Knight Rises, The (2012)',
       'Girl with the Dragon Tattoo, The (2011)',
       'Django Unchained (2012)', 'Wolf of Wall Street, The (2013)',
       'Interstellar (2014)', 'Whiplash (2014)', 'The Drop (2014)',
       'Ex Machina (2015)', 'Mad Max: Fury Road (2015)',
       'The Jinx: The Life and Deaths of Robert Durst (2015)'],
      dtype=object)

Классы в библиотеке surprise умеют принимать формат данных только определенного типа, поэтому для этого создаем новую таблицу с переименованными колонками:

In [62]:
dataset = movies_with_ratings[["title", "userId", "rating"]].rename({
    "userId": "uid",
    "title": "iid"
})

In [63]:
dataset.head()

,title,userId,rating
0,Toy Story (1995),1.0,4.0
1,Toy Story (1995),5.0,4.0
2,Toy Story (1995),7.0,4.5
3,Toy Story (1995),15.0,2.5
4,Toy Story (1995),17.0,4.5


Классы в surprise принимают данные через свой отдельный класс Dataset, в который мы отправляем данные и класс Reader с указанным диапазоном допустимых оценок:

In [64]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

Также качество нашего алгоритма мы захотим потом проверить, поэтому предлагаю поделить данные на тренировочные и тестовые, чтобы потом замерить качество на отложенной выборке:

In [65]:
trainset, testset = train_test_split(data, test_size=0.15)

Реализация классической коллаборативной фильтрации находится в классе KNNWithMeans, будем использовать её:

In [66]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...


Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Делаем предсказание на тестовом датасете:

In [67]:
test_pred = algo.test(testset)

Считаем RMSE:

In [68]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8762


0.8761563888994994

А вот так теперь можно предсказывать оценку пользователя для фильма:

In [69]:
algo.predict(uid=2, iid='Fight Club (1999)').est

3.5016975848792438

# Алгоритм ALS

In [75]:
import numpy as np
import scipy
import pandas as pd

from sklearn.model_selection import train_test_split

from implicit.als import AlternatingLeastSquares
from implicit.evaluation import mean_average_precision_at_k

In [76]:
ratings = pd.read_csv("C:\SkillFactory\SF_DataScience\Current_tasks\data\DS_ADD_1\ml-100k/u.data", sep="\t", header=None)
ratings.columns = ['user_id', 'item_id', 'rating', 'timestamp']
ratings.sort_values('timestamp', inplace=True)
ratings['score'] = (ratings['rating'] > 2).apply(int)

In [77]:
train_df, test_df = train_test_split(ratings, test_size=0.2, shuffle=False)

Чтобы обучить ALS-модель на предоставленных данных, нужно создать user-item таблицу для тренировочной и тестовой выборки. В этой таблице по строкам должны быть отложены идентификаторы всех уникальных пользователей, которые у нас есть, а по столбцам — все уникальные фильмы. То есть мы должны получить две матрицы размерности 943 x 1682. На пересечении строк и столбцов этих матриц должны быть числа, характеризующие наличие положительных и отрицательных оценок пользователей. 

Проблема заключается в том, что в тренировочную и тестовую выборку могли попасть различные пользователи и различные товары. Посмотрим на это, создав сводные таблицы для тренировочной и тестовой выборок. 

In [80]:
train_pivot = pd.pivot_table(
    train_df,
    index="user_id", 
    columns="item_id", 
    values="score"
)
test_pivot = pd.pivot_table(
    test_df,
    index="user_id", 
    columns="item_id", 
    values="score"
)

print(train_pivot.shape)
print(test_pivot.shape)

(751, 1616)
(301, 1448)


Теперь создадим сводную таблицу из таблицы rating, заполнив её ячейки нулями. Получим матрицу размером 943 x 1682. Для тех фильмов, которым пользователь выставил оценку значения, будут равны 0, для остальных — пропуску. 

In [82]:
shell = pd.pivot_table(
    ratings, 
    index="user_id", 
    columns="item_id", 
    values="score", 
    aggfunc=lambda x: 0
)
shell.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Чтобы получить тренировочную и тестовую user-item таблицы, нам осталось только сложить таблицу shell с соответствующими таблицами train_pivot и test_pivot.

Чтобы корректно обрабатывать пропущенные значения, мы трансформируем 1 в 2, а 0 — в 1. Сами пропуски заполняем нулями. В результате у нас получатся две таблицы размером 943 x 1682, в которых на пересечении пользователя и фильма стоит:

0 — если пользователь не оценил данный фильм;
1 — если пользователь оценил фильм отрицательно;
2 — если пользователь оценил фильм положительно.

In [83]:
train_pivot = shell + train_pivot
test_pivot = shell + test_pivot

train_pivot = (train_pivot + 1).fillna(0)
test_pivot = (test_pivot + 1).fillna(0)
print(train_pivot.shape)
print(test_pivot.shape)

train_pivot.head()

(943, 1682)
(943, 1682)


item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,2.0,2.0,2.0,2.0,0.0,2.0,2.0,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Финальный шаг предобработки: модели из библиотеки implicit требуют, чтобы user-item матрицы были представлены в виде разреженных матриц. Для получения разреженной матрицы используется функция csr_matrix() из модуля sparse библиотеки scipy:

In [85]:
train_pivot_sparse = scipy.sparse.csr_matrix(train_pivot.values)
test_pivot_sparse = scipy.sparse.csr_matrix(test_pivot.values)


Теперь, когда созданы тренировочная и тестовая user-item таблицы, мы готовы перейти к этапу построения модели. Обучим ALS-модель с 10-ю факторами, параметр random_state установим в значение 42.

In [86]:
model = AlternatingLeastSquares(factors=10, random_state=42)
model.fit(train_pivot_sparse)

  0%|          | 0/15 [00:00<?, ?it/s]

Чтобы сформировать рекомендации для конкретного пользователя, можно воспользоваться методом recommend(). 

Метод принимает идентификатор пользователя, для которого нужно сформировать рекомендации и его вектор оценок объектам (фильмам). Вектор оценок достаём из разреженной матрицы, на которой обучались. 

Метод возвращает индексы товаров (фильмов), которые, по мнению ALS, являются наиболее вероятными для покупки (просмотра) данным пользователем, а также веса этих товаров. 

Важный момент: индексы товаров не равны их идентификаторам! То есть метод recommend() возвращает номера столбцов из user-item таблицы, а не сами идентификаторы товаров! 

In [87]:
unique_items = np.array(train_pivot.columns)
user_id = 14
recomendations_ids, scores = model.recommend(user_id, train_pivot_sparse[user_id])
recomendations = unique_items[recomendations_ids]
print('Recomendations ids: {}'.format(recomendations_ids))
print('Recomendations for user {}: {}'.format(user_id, recomendations))

Recomendations ids: [293 116  99 275 244 287 283 150 125 596]
Recomendations for user 14: [294 117 100 276 245 288 284 151 126 597]


Итак, для пользователя с идентификатором 14 наша рекомендательная система рекомендовала фильмы под идентификаторами [294 117 100 276 245 288 284 151 126 597]. Чтобы понять, что это за фильмы, можно обратиться к таблице movies, которую мы рассматривали ранее в модуле. 

Теперь определим качество модели на всей тестовой выборке, рассчитав precision для топ 10-рекомендуемых фильмов с помощью функции mean_average_precision_at_k() из библиотеки implicit:

In [88]:
map_at10 = mean_average_precision_at_k(model, train_pivot_sparse, test_pivot_sparse, K=10)
print('Mean Average Precision at 10: {:.3f}'.format(map_at10))

  0%|          | 0/301 [00:00<?, ?it/s]

Mean Average Precision at 10: 0.087


# Модель факторизационных машин

In [90]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

c:\Users\Пользователь\AppData\Local\Programs\Python\Python310\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


Обучим факторизационные машины с 10-ю факторами, в качестве функции потерь используем logloss, параметр random_state установим в значение 42. Обучение будем производить на 30 итерациях (эпохах):

In [91]:
model = LightFM(no_components=10, loss='logistic', random_state=42)
model.fit(train_pivot_sparse, epochs=30)

Процесс построения рекомендаций для пользователей у моделей из модуля lightfm сильно отличается от того же процесса у моделей из implicit. 

Предсказание осуществляется с помощью метода predict(), который принимает на вход идентификатор пользователя, а также индексы всех объектов (фильмов); индексы обязательно начинаются от 0. Метод возвращает веса для каждого объекта, причём веса объектов отрицательные. 

Чтобы получить сами рекомендации, необходимо умножить эти веса на -1 и отсортировать их индексы по возрастанию веса. Нам нужны будут только индексы 10 фильмов с наибольшим по модулю весом. Обратившись по полученным индексам к списку идентификаторов фильмов, мы получим рекомендации для конкретного пользователя:

In [92]:
item_ids = np.arange(0, train_pivot_sparse.shape[1])
list_pred = model.predict(user_id, item_ids)
recomendations_ids = np.argsort(-list_pred)[:10]
recomendations = unique_items[recomendations_ids]
print('Recomendations for user {}: {}'.format(user_id, recomendations))

Recomendations for user 14: [ 50 294 258 100 181 288 286   1 300 121]


In [93]:
map_at10 = precision_at_k(model, test_pivot_sparse, k=10).mean()
print('Mean Average Precision at 10: {:.2f}'.format(map_at10))

Mean Average Precision at 10: 0.32
